# Мини-игра

#1. Импортируем библиотеки

In [101]:
import random
import json
from abc import ABC, abstractmethod
from enum import Enum
from typing import List, Dict, Optional, Any
import uuid
from datetime import datetime

#2. Дескрипторы и базовые классы BoundedStat и Human

BoundedStat - дескриптор для ограничения значений характеристик.

Human - базовый класс с основными характеристиками и методами.

LoggerMixin для логирования

In [102]:
class BoundedStat:
    def __init__(self, min_val: int = 0, max_val: int = 100):
        self.min_val = min_val
        self.max_val = max_val
        self.values = {}

    def __get__(self, instance, owner):
        return self.values.get(id(instance), self.min_val)

    def __set__(self, instance, value):
        if value < self.min_val:
            value = self.min_val
        elif value > self.max_val:
            value = self.max_val
        self.values[id(instance)] = value

class LoggerMixin:
    def log(self, message: str):
        timestamp = datetime.now().strftime("%H:%M:%S")
        print(f"[{timestamp}] {message}")

class Human(LoggerMixin):
    hp = BoundedStat(0, 2000)
    mp = BoundedStat(0, 500)
    strength = BoundedStat(1, 100)
    agility = BoundedStat(1, 100)
    intelligence = BoundedStat(1, 100)

    def __init__(self, name: str, level: int = 1):
        self.name = name
        self.level = level
        self._hp = 100
        self._mp = 50
        self._strength = 10
        self._agility = 10
        self._intelligence = 10
        self.max_hp = 100
        self.max_mp = 50
        self.effects = []

    @property
    def is_alive(self) -> bool:
        return self.hp > 0

    def take_damage(self, damage: int):
        self.hp -= damage

    def heal(self, amount: int):
        old_hp = self.hp
        self.hp += amount
        if self.hp > self.max_hp:
            self.hp = self.max_hp
        return self.hp - old_hp

    def add_effect(self, effect: 'Effect'):
        self.effects.append(effect)

    def process_effects(self):
        effects_to_remove = []
        for effect in self.effects:
            effect.apply(self)
            effect.duration -= 1
            if effect.duration <= 0:
                effects_to_remove.append(effect)

        for effect in effects_to_remove:
            self.effects.remove(effect)

    def get_effects_display(self) -> str:
        if not self.effects:
            return "нет"
        return ", ".join(str(effect) for effect in self.effects)

    def get_status_display(self) -> str:
        status = "Alive" if self.is_alive else "Death"
        return f"{status} {self.name} (HP: {self.hp}/{self.max_hp}, MP: {self.mp}/{self.max_mp}) | Эффекты: {self.get_effects_display()}"

    def __str__(self):
        return self.get_status_display()


#3. Система эффектов
 Базовый класс Effect и конкретные эффекты (Яд, Регенерация, Щит)

In [103]:
class EffectType(Enum):
    BUFF = "Бафф"
    DEBUFF = "Дебафф"
    DOT = "Урон_время"
    HOT = "Лечение_время"
    SHIELD = "Щит"

class Effect:
    def __init__(self, name: str, effect_type: EffectType, duration: int = 3):
        self.name = name
        self.effect_type = effect_type
        self.duration = duration

    def apply(self, target: Human):
        pass

    def __str__(self):
        return f"{self.name} ({self.effect_type.value}, {self.duration} ходов)"

class PoisonEffect(Effect):
    def __init__(self, damage_per_turn: int = 5, duration: int = 3):
        super().__init__("Яд", EffectType.DOT, duration)
        self.damage_per_turn = damage_per_turn

    def apply(self, target: Human):
        target.take_damage(self.damage_per_turn)

class RegenerationEffect(Effect):
    def __init__(self, heal_per_turn: int = 5, duration: int = 3):
        super().__init__("Регенерация", EffectType.HOT, duration)
        self.heal_per_turn = heal_per_turn

    def apply(self, target: Human):
        target.heal(self.heal_per_turn)

class ShieldEffect(Effect):
    def __init__(self, shield_amount: int = 20, duration: int = 2):
        super().__init__("Щит", EffectType.SHIELD, duration)
        self.shield_amount = shield_amount
        self.remaining_shield = shield_amount

    def apply(self, target: Human):
        pass

    def absorb_damage(self, damage: int) -> int:
        if self.remaining_shield >= damage:
            self.remaining_shield -= damage
            return 0
        else:
            remaining_damage = damage - self.remaining_shield
            self.remaining_shield = 0
            return remaining_damage

#4.  Абстрактные классы Skill и Character
основа для навыков и персонажей


In [104]:
class Skill(ABC):
    def __init__(self, name: str, mp_cost: int = 0, cooldown: int = 0):
        self.name = name
        self.mp_cost = mp_cost
        self.cooldown = cooldown
        self.current_cooldown = 0

    @abstractmethod
    def execute(self, caster: 'Character', target: Optional[Human] = None) -> bool:
        pass

    def can_use(self, caster: 'Character') -> bool:
        if caster.mp < self.mp_cost:
            return False
        if self.current_cooldown > 0:
            return False
        return True

    def get_status_icon(self) -> str:
        return "Succesful" if self.current_cooldown == 0 else "Wait"

    def start_cooldown(self):
        self.current_cooldown = self.cooldown

    def reduce_cooldown(self):
        if self.current_cooldown > 0:
            self.current_cooldown -= 1

    def __str__(self):
        status = self.get_status_icon()
        return f"{self.name} ({self.mp_cost} MP) {status}"


class BasicAttack(Skill):
    def __init__(self):
        super().__init__("Базовая атака", 0, 0)

    def execute(self, caster: 'Character', target: Optional[Human] = None) -> bool:
        if not target:
            return False

        damage = caster.strength + random.randint(1, 5)
        if random.random() < 0.2:
            damage *= 2
            caster.log(f"Критический урон! {caster.name} атакует {target.name} на {damage} урона!")
        else:
            caster.log(f"{caster.name} атакует {target.name} на {damage} урона")

        actual_damage = damage
        for effect in target.effects:
            if isinstance(effect, ShieldEffect):
                actual_damage = effect.absorb_damage(damage)
                if actual_damage < damage:
                    caster.log(f"Щит поглотил {damage - actual_damage} урона")
                    if actual_damage == 0:
                        break

        target.take_damage(actual_damage)
        return True

class Character(Human, ABC):

    def __init__(self, name: str, level: int = 1):
        super().__init__(name, level)
        self.skills = [BasicAttack()]

    @abstractmethod
    def use_skill(self, skill_index: int, target: Optional[Human] = None) -> bool:
        pass

    def basic_attack(self, target: Human) -> bool:
        return self.skills[0].execute(self, target)

    def reduce_skills_cooldown(self):
        for skill in self.skills:
            skill.reduce_cooldown()

#5. Классы персонажей воина, мага и лекаря

  Warrior - класс воина (сила, HP, MP)

  Mage - класс мага (интеллект, эффекты, магические атаки)

  Healer - класс лекаря (лечение, очистка эффектов)

In [105]:
class Warrior(Character):
    def __init__(self, name: str, level: int = 1):
        super().__init__(name, level)
        self._strength = 25
        self._agility = 12
        self._intelligence = 8
        self._hp = 170
        self._mp = 35
        self.max_hp = 170
        self.max_mp = 35

        self.skills.append(PowerStrike())
        self.skills.append(WhirlwindAttack())
        self.skills.append(Taunt())

    def use_skill(self, skill_index: int, target: Optional[Human] = None) -> bool:
        if skill_index >= len(self.skills):
            return False

        skill = self.skills[skill_index]
        if skill.can_use(self):
            self.mp -= skill.mp_cost
            result = skill.execute(self, target)
            if result:
                skill.start_cooldown()
            return result
        return False

class Mage(Character):
    def __init__(self, name: str, level: int = 1):
        super().__init__(name, level)
        self._strength = 8
        self._agility = 12
        self._intelligence = 25
        self._hp = 90
        self._mp = 115
        self.max_hp = 90
        self.max_mp = 115

        self.skills.append(Fireball())
        self.skills.append(FrostBolt())
        self.skills.append(ArcaneShield())

    def use_skill(self, skill_index: int, target: Optional[Human] = None) -> bool:
        if skill_index >= len(self.skills):
            return False

        skill = self.skills[skill_index]
        if skill.can_use(self):
            self.mp -= skill.mp_cost
            result = skill.execute(self, target)
            if result:
                skill.start_cooldown()
            return result
        return False

class Healer(Character):
    def __init__(self, name: str, level: int = 1):
        super().__init__(name, level)
        self._strength = 10
        self._agility = 14
        self._intelligence = 22
        self._hp = 112
        self._mp = 92
        self.max_hp = 112
        self.max_mp = 92

        self.skills.append(Heal())
        self.skills.append(Purify())
        self.skills.append(Blessing())

    def use_skill(self, skill_index: int, target: Optional[Human] = None) -> bool:
        if skill_index >= len(self.skills):
            return False

        skill = self.skills[skill_index]
        if skill.can_use(self):
            self.mp -= skill.mp_cost
            result = skill.execute(self, target)
            if result:
                skill.start_cooldown()
            return result
        return False

# 6.Конкретные навыки

PowerStrike, WhirlwindAttack - атаки воина

Fireball, FrostBolt - магические атаки

Heal, Blessing - навыки лечения

In [106]:
class PowerStrike(Skill):
    def __init__(self):
        super().__init__("power_strike", 15, 2)

    def execute(self, caster: Character, target: Optional[Human] = None) -> bool:
        if not target:
            return False

        damage = caster.strength * 2 + random.randint(10, 20)
        print(f"{caster.name} использует Мощный удар по {target.name} и наносит {damage} урона!")
        target.take_damage(damage)
        return True

class Taunt(Skill):
    def __init__(self):
        super().__init__("taunt", 10, 3)

    def execute(self, caster: Character, target: Optional[Human] = None) -> bool:
        if not target:
            return False

        print(f"{caster.name} провоцирует {target.name}!")
        return True

class FrostBolt(Skill):
    def __init__(self):
        super().__init__("frost_bolt", 15, 1)

    def execute(self, caster: Character, target: Optional[Human] = None) -> bool:
        if not target:
            return False

        damage = caster.intelligence + random.randint(10, 18)
        print(f"{caster.name} бросает Ледяную стрелу в {target.name} и наносит {damage} урона!")
        target.take_damage(damage)
        return True

class ArcaneShield(Skill):
    def __init__(self):
        super().__init__("arcane_shield", 30, 4)

    def execute(self, caster: Character, target: Optional[Human] = None) -> bool:
        if not target:
            target = caster

        print(f"{caster.name} накладывает Магический щит на {target.name}!")
        target.add_effect(ShieldEffect(50, 3))
        return True

class Purify(Skill):
    def __init__(self):
        super().__init__("purify", 15, 2)

    def execute(self, caster: Character, target: Optional[Human] = None) -> bool:
        if not target:
            return False

        print(f"{caster.name} очищает {target.name} от негативных эффектов!")
        target.effects = [eff for eff in target.effects if not isinstance(eff, (PoisonEffect, BurnEffect))]
        return True

class Blessing(Skill):
    def __init__(self):
        super().__init__("blessing", 25, 3)

    def execute(self, caster: Character, target: Optional[Human] = None) -> bool:
        if not target:
            return False

        print(f"{caster.name} благословляет {target.name}!")
        target.add_effect(RegenerationEffect(20, 3))
        return True

# 7.Дополнительные навыки персонажей:
- WhirlwindAttack - массовая атака воина по всем врагам
- ShieldSlam - удар щитом с гарантированным уроном
- Fireball - огненный шар с шансом поджечь цель
- Heal/GreaterHeal - лечение одного союзника
- RegenerationSpell - регенерация на несколько ходов

In [107]:
class WhirlwindAttack(Skill):
    def __init__(self):
        super().__init__("Вихрь атаки", 15, 3)

    def execute(self, caster: Character, target: Optional[Human] = None) -> bool:
        battle = Battle.get_current_battle()
        if not battle:
            return False

        caster.log(f"{caster.name} использует Вихрь атаки!")
        for enemy in battle.enemies:
            if enemy.is_alive:
                damage = caster.strength + random.randint(3, 8)
                enemy.take_damage(damage)
                caster.log(f"{enemy.name} получает {damage} урона!")
        return True

class ShieldSlam(Skill):
    def __init__(self):
        super().__init__("Удар щитом", 10, 2)

    def execute(self, caster: Character, target: Optional[Human] = None) -> bool:
        if not target:
            return False

        damage = caster.strength + 10
        caster.log(f"{caster.name} использует Удар щитом по {target.name}!")
        target.take_damage(damage)

        return True

class Fireball(Skill):
    def __init__(self):
        super().__init__("fireball", 20, 2)

    def execute(self, caster: Character, target: Optional[Human] = None) -> bool:
        if not target:
            return False

        damage = caster.intelligence * 2 + random.randint(15, 25)
        print(f"{caster.name} бросает Огненный шар в {target.name} и наносит {damage} урона!")
        target.take_damage(damage)

        if random.random() < 0.4:
            print(f"{target.name} начинает гореть!")
            target.add_effect(BurnEffect(20, 2))
        return True

class Heal(Skill):
    def __init__(self):
        super().__init__("Лечение", 15, 1)

    def execute(self, caster: Character, target: Optional[Human] = None) -> bool:
        if not target:
            return False

        heal_amount = caster.intelligence + random.randint(5, 10)
        caster.log(f"{caster.name} лечит {target.name}!")
        target.heal(heal_amount)
        return True

class GreaterHeal(Skill):
    def __init__(self):
        super().__init__("Сильное лечение", 25, 2)

    def execute(self, caster: Character, target: Optional[Human] = None) -> bool:
        if not target:
            return False

        heal_amount = caster.intelligence * 2 + random.randint(10, 20)
        caster.log(f"{caster.name} использует Сильное лечение на {target.name}!")
        target.heal(heal_amount)
        return True

class RegenerationSpell(Skill):
    def __init__(self):
        super().__init__("Регенерация", 20, 3)

    def execute(self, caster: Character, target: Optional[Human] = None) -> bool:
        if not target:
            return False

        caster.log(f"{caster.name} накладывает Регенерацию на {target.name}!")
        target.add_effect(RegenerationEffect(8, 4))
        return True

#8.Класс босса и его стратегии
Boss - класс босса с фазами

Меняет поведение в зависимости от здоровья



In [108]:
class BossStrategy(ABC):
    @abstractmethod
    def execute(self, boss: 'Boss', targets: List[Human]):
        pass

class AggressiveStrategy(BossStrategy):
    def execute(self, boss: 'Boss', targets: List[Human]):
        alive_targets = [t for t in targets if t.is_alive]
        if alive_targets:
            target = random.choice(alive_targets)
            boss.basic_attack(target)

class AOEStrategy(BossStrategy):
    def execute(self, boss: 'Boss', targets: List[Human]):
        boss.log(f"{boss.name} использует массовую атаку!")
        for target in targets:
            if target.is_alive:
                damage = 15 + random.randint(5, 10)
                target.take_damage(damage)

class DebuffStrategy(BossStrategy):
    def execute(self, boss: 'Boss', targets: List[Human]):
        alive_targets = [t for t in targets if t.is_alive]
        if alive_targets:
            target = random.choice(alive_targets)
            boss.log(f"{boss.name} накладывает яд на {target.name}!")
            target.add_effect(PoisonEffect(10, 3))

class Boss(Character):
    def __init__(self, name: str, level: int = 10):
        super().__init__(name, level)
        self._strength = 35
        self._agility = 18
        self._intelligence = 28
        self._hp = 1000
        self._mp = 400
        self.max_hp = 1000
        self.max_mp = 400

        self.strategies = [
            AggressiveStrategy(),
            AOEStrategy(),
            DebuffStrategy()
        ]
        self.current_strategy_index = 0

    def use_skill(self, skill_index: int, target: Optional[Human] = None) -> bool:
        return True

    def make_turn(self, targets: List[Human]):
        if not self.is_alive:
            return

        hp_percent = self.hp / self.max_hp
        if hp_percent < 0.3:
            self.current_strategy_index = 2
        elif hp_percent < 0.6:
            self.current_strategy_index = 1
        else:
            self.current_strategy_index = 0

        strategy = self.strategies[self.current_strategy_index]
        strategy.execute(self, targets)


#9. Система порядка ходов и основной класс битвы
Содержит TurnOrder - итератор для порядка ходов

Класс Battle - логика битвы

In [109]:
class TurnOrder:
    def __init__(self, participants: List[Human]):
        self.participants = participants
        self.index = 0

    def __iter__(self):
        sorted_participants = sorted(
            self.participants,
            key=lambda x: x.agility,
            reverse=True
        )
        return iter([p for p in sorted_participants if p.is_alive])

    def __next__(self):
        if self.index >= len(self.participants):
            self.index = 0
            raise StopIteration
        participant = self.participants[self.index]
        self.index += 1
        return participant

class Battle(LoggerMixin):
    _current_battle = None

    def __init__(self, party: List[Character], boss: Boss):
        self.party = party
        self.boss = boss
        self.round = 0
        self.turn_order = None
        Battle._current_battle = self

    @classmethod
    def get_current_battle(cls):
        return cls._current_battle

    @property
    def all_participants(self) -> List[Human]:
        return self.party + [self.boss]

    @property
    def enemies(self) -> List[Human]:
        return [self.boss]

    def is_battle_over(self) -> bool:
        party_alive = any(character.is_alive for character in self.party)
        boss_alive = self.boss.is_alive

        if not party_alive:
            self.log("Пати уничтожена! Босс побеждает!")
            return True
        elif not boss_alive:
            self.log("Босс повержен! Пати побеждает!")
            return True
        return False

    def start_battle(self):
        self.log("Начало боя!")
        self.log(f"Пати: {[char.name for char in self.party]}")
        self.log(f"Противник: {self.boss.name}")

        while not self.is_battle_over():
            self.round += 1
            self.log(f"\n--- Раунд {self.round} ---")
            self.execute_round()

        self.log("\n--- БОЙ ЗАВЕРШЕН ---")

    def execute_round(self):
        for participant in self.all_participants:
            if participant.is_alive:
                participant.process_effects()

        turn_order = TurnOrder(self.all_participants)

        for participant in turn_order:
            if self.is_battle_over():
                break

            if not participant.is_alive:
                continue

            self.log(f"\nХод {participant.name}:")

            if isinstance(participant, Boss):
                participant.make_turn(self.party)
            else:
                self.player_turn(participant)

            if isinstance(participant, Character):
                participant.reduce_skills_cooldown()

    def player_turn(self, player: Character):
        if not player.is_alive:
            return

        if isinstance(player, Healer):
            wounded_allies = [ally for ally in self.party if ally.is_alive and ally.hp < ally.max_hp * 0.7]
            if wounded_allies and random.random() < 0.7:
                target = random.choice(wounded_allies)
                for i in range(1, len(player.skills)):
                    if player.use_skill(i, target):
                        return
            player.basic_attack(self.boss)

        elif isinstance(player, Mage):
            if random.random() < 0.6 and len(player.skills) > 1:
                skill_index = random.randint(1, len(player.skills) - 1)
                if player.use_skill(skill_index, self.boss):
                    return
            player.basic_attack(self.boss)

        else:
            if random.random() < 0.4 and len(player.skills) > 1:
                skill_index = random.randint(1, len(player.skills) - 1)
                if player.use_skill(skill_index, self.boss):
                    return
            player.basic_attack(self.boss)

#10. Инвентарь и предметы
Зелья здоровья и маны

In [110]:
class Item:
    def __init__(self, name: str, item_type: str):
        self.name = name
        self.item_type = item_type

    def use(self, target: Human) -> bool:
        return False

class HealthPotion(Item):
    def __init__(self):
        super().__init__("Зелье здоровья", "potion")

    def use(self, target: Human) -> bool:
        heal_amount = 50
        target.heal(heal_amount)
        target.log(f"{target.name} использует {self.name} и восстанавливает {heal_amount} HP!")
        return True

class ManaPotion(Item):
    def __init__(self):
        super().__init__("Зелье маны", "potion")

    def use(self, target: Human) -> bool:
        mana_amount = 30
        old_mp = target.mp
        target.mp += mana_amount
        actual_mana = target.mp - old_mp
        target.log(f"{target.name} использует {self.name} и восстанавливает {actual_mana} MP!")
        return True

class Inventory:
    def __init__(self):
        self.items = []

    def add_item(self, item: Item):
        self.items.append(item)

    def use_item(self, item_index: int, target: Human) -> bool:
        if 0 <= item_index < len(self.items):
            item = self.items.pop(item_index)
            return item.use(target)
        return False

#11. Демо-версия игры
 Автоматическая битва

In [111]:
def demo_game():
    print("Запуск игры 'Пати против Босса'")
    print("=" * 50)

    warrior = Warrior("Боромир", 3)
    mage = Mage("Гэндальф", 4)
    healer = Healer("Элронд", 3)

    party = [warrior, mage, healer]

    boss = Boss("Дракон Урлок", 5)

    warrior_inventory = Inventory()
    warrior_inventory.add_item(HealthPotion())
    warrior_inventory.add_item(HealthPotion())

    mage_inventory = Inventory()
    mage_inventory.add_item(ManaPotion())

    battle = Battle(party, boss)
    battle.start_battle()


#12. Юнит-тесты для проверки функциональности

In [112]:
import unittest

class TestGame(unittest.TestCase):

    def test_character_creation(self):
        warrior = Warrior("Тестовый воин")
        self.assertEqual(warrior.name, "Тестовый воин")
        self.assertTrue(warrior.is_alive)
        self.assertEqual(warrior.strength, 20)

    def test_damage_and_heal(self):
        character = Warrior("Тест")
        initial_hp = character.hp

        character.take_damage(30)
        self.assertEqual(character.hp, initial_hp - 30)

        character.heal(20)
        self.assertEqual(character.hp, initial_hp - 10)

    def test_effects(self):
        character = Warrior("Тест")
        poison = PoisonEffect(5, 2)

        character.add_effect(poison)
        self.assertEqual(len(character.effects), 1)

        character.process_effects()
        self.assertEqual(character.hp, character.max_hp - 5)

    def test_skill_usage(self):
        mage = Mage("Тестовый маг")
        boss = Boss("Тестовый босс")

        initial_mp = mage.mp
        success = mage.use_skill(1, boss)  # Fireball

        self.assertTrue(success)
        self.assertEqual(mage.mp, initial_mp - mage.skills[1].mp_cost)

In [113]:
def run_tests():
    suite = unittest.TestLoader().loadTestsFromTestCase(TestGame)
    runner = unittest.TextTestRunner(verbosity=2)
    result = runner.run(suite)

# 13.Система сохранения и загрузки в JSON

In [114]:
class GameState:
    @staticmethod
    def save_to_json(battle: Battle, filename: str = "game_state.json"):
        state = {
            "round": battle.round,
            "party": [],
            "boss": {},
            "timestamp": datetime.now().isoformat()
        }

        for character in battle.party:
            char_data = {
                "name": character.name,
                "class": character.__class__.__name__,
                "level": character.level,
                "hp": character.hp,
                "mp": character.mp,
                "max_hp": character.max_hp,
                "max_mp": character.max_mp,
                "effects": [effect.name for effect in character.effects]
            }
            state["party"].append(char_data)

        state["boss"] = {
            "name": battle.boss.name,
            "hp": battle.boss.hp,
            "mp": battle.boss.mp,
            "max_hp": battle.boss.max_hp,
            "max_mp": battle.boss.max_mp,
            "effects": [effect.name for effect in battle.boss.effects]
        }

        with open(filename, 'w', encoding='utf-8') as f:
            json.dump(state, f, indent=2, ensure_ascii=False)

        print(f"Состояние игры сохранено в {filename}")

    @staticmethod
    def load_from_json(filename: str = "game_state.json") -> Dict[str, Any]:
        try:
            with open(filename, 'r', encoding='utf-8') as f:
                return json.load(f)
        except FileNotFoundError:
            print(f"Файл {filename} не найден")
            return {}

In [115]:
def demo_save_load():
    print("\n--- Демо сохранения состояния ---")

    warrior = Warrior("Арагорн", 3)
    mage = Mage("Саруман", 4)
    boss = Boss("Балрог", 5)

    battle = Battle([warrior, mage], boss)

    warrior.take_damage(30)
    mage.take_damage(20)
    boss.take_damage(50)

    GameState.save_to_json(battle, "demo_save.json")

    loaded_state = GameState.load_from_json("demo_save.json")
    if loaded_state:
        print("Загруженное состояние:")
        print(json.dumps(loaded_state, indent=2, ensure_ascii=False))


#14. Дополнительные эффекты (BurnEffect), усиленная стратегия босса

In [116]:
print("Все классы и системы реализованы.")
print("Для запуска демо-игры выполните: demo_game()")

Все классы и системы реализованы.
Для запуска демо-игры выполните: demo_game()


In [117]:
class BurnEffect(Effect):
    def __init__(self, damage_per_turn: int = 20, duration: int = 2):
        super().__init__("Горение", EffectType.DOT, duration)
        self.damage_per_turn = damage_per_turn

    def apply(self, target: Human):
        if target.is_alive:
            target.take_damage(self.damage_per_turn)

#15.Интерфейс игры -
 Интерактивный выбор действий, создание партии, выбор сложности

In [118]:
class AOEStrategy(BossStrategy):
    def execute(self, boss: 'Boss', targets: List[Human]):
        print(f"{boss.name} использует Огненный взрыв!")
        for target in targets:
            if target.is_alive:
                damage = 120
                print(f"{target.name} получает {damage} урона и начинает гореть!")
                target.take_damage(damage)
                target.add_effect(BurnEffect(20, 2))

In [119]:
class GameInterface:
    @staticmethod
    def get_seed() -> Optional[int]:
        seed_input = input("\nВведите seed для случайной генерации (или оставьте пустым): ")
        if seed_input.strip():
            try:
                return int(seed_input)
            except ValueError:
                return hash(seed_input)
        return None


    @staticmethod
    def print_header(text: str):
        print(f"\n{text}")
        print("=" * 50)

    @staticmethod
    def print_round(round_num: int):
        print(f"\n==================================================")
        print(f"РАУНД {round_num}")
        print(f"==================================================")

    @staticmethod
    def create_party() -> List[Character]:
        """Создание партии персонажей"""
        print("ДОБРО ПОЖАЛОВАТЬ В МИНИ-ИГРУ!")
        print("Сразитесь с могучим боссом в пошаговом бою!")
        print("Создание партии")

        party = []
        for i in range(3):
            print(f"\n Персонаж {i + 1}:")
            print("1. Воин (сильный, много HP)")
            print("2. Маг (интеллект, магические атаки)")
            print("3. Лекарь (лечение, поддержка)")

            while True:
                try:
                    choice = int(input("Выберите класс: "))
                    if choice in [1, 2, 3]:
                        break
                    print("Неверный выбор! Введите 1, 2 или 3")
                except ValueError:
                    print("Введите число!")

            class_names = {1: "воин", 2: "маг", 3: "лекарь"}
            class_types = {1: Warrior, 2: Mage, 3: Healer}

            name = input(f"Введите имя {class_names[choice]}: ").strip()
            character = class_types[choice](name)
            party.append(character)
            print(f"{class_names[choice].capitalize()} {name} добавлен в партию!")

        return party

    @staticmethod
    def select_difficulty() -> int:
        print("\n Выберите сложность (1-легкая, 2-нормальная, 3-сложная): ")
        while True:
            try:
                difficulty = int(input())
                if difficulty in [1, 2, 3]:
                    return difficulty
                print("Неверный выбор! Введите 1, 2 или 3")
            except ValueError:
                print("Введите число!")

    @staticmethod
    def get_seed() -> Optional[int]:
        seed_input = input("\n Введите seed для случайной генерации (или оставьте пустым): ")
        if seed_input.strip():
            try:
                return int(seed_input)
            except ValueError:
                return hash(seed_input)
        return None

    @staticmethod
    def show_battle_start(party: List[Character], boss: Boss):
        print("\n Начало битвы!")
        print(f" Партия: {[char.name for char in party]}")
        print(f"Босс {boss.name} (Уровень {boss.level})")

In [120]:
class InteractiveBattle(Battle):
    def __init__(self, party, boss):
        super().__init__(party, boss)
        self.current_tactic = None
        self.next_tactic = None
        self.tactic_bonus = 1.0
        self.max_rounds = RoundManager.MAX_ROUNDS

        for hero in self.party:
            hero.hp = hero.max_hp
            hero.mp = hero.max_mp

    def start_battle(self):
        self.log("Начало боя с раундами!")
        while not self.is_battle_over() and self.round < self.max_rounds:
            self.round += 1
            GameInterface.print_round(self.round)
            self.apply_tactic_effect()
            self.execute_round()
            self.clear_tactic()

            if not self.is_battle_over() and self.round < self.max_rounds:
                RoundManager.choose_tactic(self)

        if not self.is_battle_over():
            self.finish_by_round_limit()

    def apply_tactic_effect(self):
        if not self.next_tactic:
            return
        self.current_tactic = self.next_tactic
        self.next_tactic = None

        if self.current_tactic == "оборона":
            for member in self.party:
                member.add_effect(ShieldEffect(int(member.max_hp * 0.2), 1))
                print(f"{member.name} получает щит (тактика Оборона)")
        elif self.current_tactic == "атака":
            self.tactic_bonus = 1.25
            print("Тактика Атака активна: урон увеличен на 25%!")

    def clear_tactic(self):
        self.current_tactic = None
        self.tactic_bonus = 1.0

    def execute_round(self):
        for participant in self.all_participants:
            if participant.is_alive:
                participant.process_effects()

        for participant in TurnOrder(self.all_participants):
            if self.is_battle_over():
                break
            if not participant.is_alive:
                continue

            print(f"\n--- Ход {participant.name} ---")

            if isinstance(participant, Boss):
                participant.make_turn(self.party)
            else:
                self.player_turn(participant)

            if isinstance(participant, Character):
                participant.reduce_skills_cooldown()

            self.show_current_status()

    def player_turn(self, player: Character):
        if not player.is_alive:
            return

        print(f"\n--- Ход {player.name} ---")
        print("1. Базовая атака")
        print("2. Использовать навык")

        for i, skill in enumerate(player.skills[1:], 3):
            status = "Доступен" if skill.can_use(player) else f"Ожидание ({skill.current_cooldown})"
            print(f"{i}. {skill.name} ({skill.mp_cost} MP) {status}")

        while True:
            try:
                choice = int(input("Выберите действие: "))

                if choice == 1:
                    target = self.select_target(player, "атаковать")
                    if target:
                        base_damage = player.strength + random.randint(3, 8)
                        actual_damage = int(base_damage * self.tactic_bonus)
                        target.take_damage(actual_damage)
                        print(f"{player.name} атакует {target.name} и наносит {actual_damage} урона!")
                    break

                elif choice >= 3 and choice - 1 < len(player.skills):
                    skill_index = choice - 1
                    skill = player.skills[skill_index]

                    if not skill.can_use(player):
                        print("Навык недоступен!")
                        continue

                    if isinstance(player, Healer):
                        target = self.select_target(player, f"использовать {skill.name} на", include_allies=True)
                    else:
                        target = self.select_target(player, f"использовать {skill.name} на")

                    if target and player.use_skill(skill_index, target):
                        break
                    else:
                        print("Не удалось применить навык!")
                else:
                    print("Неверный выбор!")

            except ValueError:
                print("Введите число!")

    def select_target(self, player: Character, action: str, include_allies: bool = False):
        print(f"\nВыберите цель для {action}:")
        targets = []

        for i, enemy in enumerate(self.enemies, 1):
            if enemy.is_alive:
                print(f"{i}. {enemy.name} (HP: {enemy.hp}/{enemy.max_hp})")
                targets.append(enemy)

        if include_allies:
            start_index = len(targets) + 1
            for i, ally in enumerate(self.party, start_index):
                if ally.is_alive and ally != player:
                    print(f"{i}. {ally.name} (HP: {ally.hp}/{ally.max_hp})")
                    targets.append(ally)

        while True:
            try:
                choice = int(input("Номер цели: "))
                if 1 <= choice <= len(targets):
                    return targets[choice - 1]
                else:
                    print("Неверный выбор цели!")
            except ValueError:
                print("Введите число!")

    def show_current_status(self):
        print(f"\nТекущее состояние:")
        for participant in self.all_participants:
            print(f"  {participant}")

    def finish_by_round_limit(self):
        print("\n=== Лимит раундов достигнут! ===")
        team_hp = sum(max(0, char.hp) for char in self.party)
        boss_hp = max(0, self.boss.hp)
        print(f"HP пати: {team_hp} | HP босса: {boss_hp}")
        if team_hp > boss_hp:
            print("Пати побеждает по очкам!")
        elif boss_hp > team_hp:
            print("Босс побеждает по очкам!")
        else:
            print("Ничья!")

#16. Интерактивная битва: выбор действий игроком, целеполагание

In [130]:
def interactive_game():
    party = GameInterface.create_party()
    difficulty = GameInterface.select_difficulty()

    seed = GameInterface.get_seed()
    if seed is not None:
        random.seed(seed)
        print(f"Установлен seed: {seed}")

    boss_names = ["Дракон Повелитель Тьмы", "Древний Лич", "Владыка Бездны"]
    boss_levels = {1: 8, 2: 10, 3: 12}
    boss_hp = {1: 800, 2: 1000, 3: 1200}

    boss_name = random.choice(boss_names)
    boss = Boss(boss_name, boss_levels[difficulty])
    boss.max_hp = boss_hp[difficulty]
    boss.hp = boss_hp[difficulty]

    print(f"Создан босс: {boss.name} (Уровень {boss.level})")

    GameInterface.show_battle_start(party, boss)

    for character in party:
        character.hp = character.max_hp
        character.mp = character.max_mp

    battle = InteractiveBattle(party, boss)
    battle.start_battle()

In [131]:
class PlayerTactic:
    def __init__(self, name, description, apply_func):
        self.name = name
        self.description = description
        self.apply_func = apply_func

    def apply(self, battle: 'InteractiveBattle'):
        self.apply_func(battle)


class RoundManager:
    MAX_ROUNDS = 5

    @staticmethod
    def list_tactics():
        return [
            ("Оборона", "Щит для всей пати: уменьшает входящий урон на следующий раунд."),
            ("Атака", "Увеличивает урон пати на следующий раунд."),
            ("Восстановление", "Мгновенно восстанавливает 15% HP всей пати.")
        ]

    @staticmethod
    def choose_tactic(battle: 'InteractiveBattle'):
        print("\nВыберите тактику на следующий раунд:")
        for i, (name, desc) in enumerate(RoundManager.list_tactics(), 1):
            print(f"{i}. {name} — {desc}")

        while True:
            try:
                choice = int(input("Ваш выбор: "))
                if 1 <= choice <= 3:
                    tactic = RoundManager.list_tactics()[choice - 1][0].lower()
                    battle.next_tactic = tactic
                    print(f"\nВы выбрали тактику: {tactic.capitalize()}")

                    if tactic == "восстановление":
                        for m in battle.party:
                            healed = int(m.max_hp * 0.15)
                            m.heal(healed)
                            print(f"{m.name} восстановил {healed} HP.")
                    break
                else:
                    print("Введите число от 1 до 3.")
            except ValueError:
                print("Введите корректное число.")


class GameInterface:
    @staticmethod
    def get_seed() -> Optional[int]:
        seed_input = input("\nВведите seed для случайной генерации (или оставьте пустым): ")
        if seed_input.strip():
            try:
                return int(seed_input)
            except ValueError:
                return hash(seed_input)
        return None

    @staticmethod
    def print_header(text: str):
        print(f"\n{text}")
        print("=" * 50)

    @staticmethod
    def print_round(round_num: int):
        print(f"\n==================================================")
        print(f"РАУНД {round_num}")
        print(f"==================================================")

    @staticmethod
    def create_party() -> List[Character]:
        print("ДОБРО ПОЖАЛОВАТЬ В МИНИ-ИГРУ!")
        print("Сразитесь с могучим боссом в пошаговом бою!")
        print("Создание партии")

        party = []
        for i in range(3):
            print(f"\nПерсонаж {i + 1}:")
            print("1. Воин (сильный, много HP)")
            print("2. Маг (интеллект, магические атаки)")
            print("3. Лекарь (лечение, поддержка)")

            while True:
                try:
                    choice = int(input("Выберите класс: "))
                    if choice in [1, 2, 3]:
                        break
                    print("Неверный выбор! Введите 1, 2 или 3")
                except ValueError:
                    print("Введите число!")

            name = input("Введите имя: ").strip()
            if choice == 1:
                character = Warrior(name)
                print(f"Воин {name} добавлен в партию!")
            elif choice == 2:
                character = Mage(name)
                print(f"Маг {name} добавлен в партию!")
            else:
                character = Healer(name)
                print(f"Лекарь {name} добавлен в партию!")

            party.append(character)

        return party

    @staticmethod
    def select_difficulty() -> int:
        print("\nВыберите сложность (1-легкая, 2-нормальная, 3-сложная): ")
        while True:
            try:
                difficulty = int(input())
                if difficulty in [1, 2, 3]:
                    return difficulty
                print("Неверный выбор! Введите 1, 2 или 3")
            except ValueError:
                print("Введите число!")

    @staticmethod
    def show_battle_start(party: List[Character], boss: Boss):
        print("\nНачало битвы!")
        print(f"Партия: {[char.name for char in party]}")
        print(f"Босс: {boss.name} (Уровень {boss.level})")


class InteractiveBattle(Battle):
    def __init__(self, party, boss):
        super().__init__(party, boss)
        self.current_tactic = None
        self.next_tactic = None
        self.tactic_bonus = 1.0
        self.max_rounds = 5

    def start_battle(self):
        print("Начало битвы с раундами!")
        GameInterface.show_battle_start(self.party, self.boss)

        while not self.is_battle_over() and self.round < self.max_rounds:
            self.round += 1
            GameInterface.print_round(self.round)
            self.apply_tactic_effect()
            self.execute_round()
            self.clear_tactic()

            if not self.is_battle_over() and self.round < self.max_rounds:
                RoundManager.choose_tactic(self)

        if not self.is_battle_over():
            self.finish_by_round_limit()
        else:
            self.log("\n--- БОЙ ЗАВЕРШЕН ---")

    def apply_tactic_effect(self):
        if not self.next_tactic:
            return
        self.current_tactic = self.next_tactic
        self.next_tactic = None

        if self.current_tactic == "оборона":
            for member in self.party:
                member.add_effect(ShieldEffect(int(member.max_hp * 0.2), 1))
                print(f"{member.name} получает щит (тактика Оборона)")
        elif self.current_tactic == "атака":
            self.tactic_bonus = 1.25
            print("Тактика Атака активна: урон увеличен на 25%!")

    def clear_tactic(self):
        self.current_tactic = None
        self.tactic_bonus = 1.0

    def execute_round(self):
        for participant in self.all_participants:
            if participant.is_alive:
                participant.process_effects()

        for participant in TurnOrder(self.all_participants):
            if self.is_battle_over():
                break
            if not participant.is_alive:
                continue

            print(f"\n--- Ход {participant.name} ---")

            if isinstance(participant, Boss):
                participant.make_turn(self.party)
            else:
                self.player_turn(participant)

            if isinstance(participant, Character):
                participant.reduce_skills_cooldown()

            self.show_current_status()

    def player_turn(self, player: Character):
        if not player.is_alive:
            return

        print(f"\n--- Ход {player.name} ---")
        print("1. Базовая атака")
        print("2. Использовать навык")

        for i, skill in enumerate(player.skills[1:], 3):
            status = "Доступен" if skill.can_use(player) else f"Ожидание ({skill.current_cooldown})"
            print(f"{i}. {skill.name} ({skill.mp_cost} MP) {status}")

        while True:
            try:
                choice = int(input("Выберите действие: "))

                if choice == 1:
                    target = self.select_target(player, "атаковать")
                    if target:
                        base_damage = player.strength + random.randint(3, 8)
                        actual_damage = int(base_damage * self.tactic_bonus)
                        target.take_damage(actual_damage)
                        print(f"{player.name} атакует {target.name} и наносит {actual_damage} урона!")
                    break

                elif choice >= 3 and choice - 1 < len(player.skills):
                    skill_index = choice - 1
                    skill = player.skills[skill_index]

                    if not skill.can_use(player):
                        print("Навык недоступен!")
                        continue

                    if isinstance(player, Healer):
                        target = self.select_target(player, f"использовать {skill.name} на", include_allies=True)
                    else:
                        target = self.select_target(player, f"использовать {skill.name} на")

                    if target and player.use_skill(skill_index, target):
                        break
                    else:
                        print("Не удалось применить навык!")
                else:
                    print("Неверный выбор!")

            except ValueError:
                print("Введите число!")

    def select_target(self, player: Character, action: str, include_allies: bool = False):
        print(f"\nВыберите цель для {action}:")
        targets = []

        for i, enemy in enumerate(self.enemies, 1):
            if enemy.is_alive:
                print(f"{i}. {enemy.name} (HP: {enemy.hp}/{enemy.max_hp})")
                targets.append(enemy)

        if include_allies:
            start_index = len(targets) + 1
            for i, ally in enumerate(self.party, start_index):
                if ally.is_alive and ally != player:
                    print(f"{i}. {ally.name} (HP: {ally.hp}/{ally.max_hp})")
                    targets.append(ally)

        while True:
            try:
                choice = int(input("Номер цели: "))
                if 1 <= choice <= len(targets):
                    return targets[choice - 1]
                else:
                    print("Неверный выбор цели!")
            except ValueError:
                print("Введите число!")

    def show_current_status(self):
        print(f"\nТекущее состояние:")
        for participant in self.all_participants:
            print(f"  {participant}")

    def finish_by_round_limit(self):
        print("\n=== Лимит раундов достигнут! ===")
        team_hp = sum(max(0, char.hp) for char in self.party)
        boss_hp = max(0, self.boss.hp)
        print(f"HP пати: {team_hp} | HP босса: {boss_hp}")
        if team_hp > boss_hp:
            print("Пати побеждает по очкам!")
        elif boss_hp > team_hp:
            print("Босс побеждает по очкам!")
        else:
            print("Ничья!")



# 17. Запуск интерактивной версии с кастомизацией

In [134]:
if __name__ == "__main__":
    interactive_game()

ДОБРО ПОЖАЛОВАТЬ В МИНИ-ИГРУ!
Сразитесь с могучим боссом в пошаговом бою!
Создание партии

Персонаж 1:
1. Воин (сильный, много HP)
2. Маг (интеллект, магические атаки)
3. Лекарь (лечение, поддержка)
Выберите класс: 1
Введите имя: Рыцарь
Воин Рыцарь добавлен в партию!

Персонаж 2:
1. Воин (сильный, много HP)
2. Маг (интеллект, магические атаки)
3. Лекарь (лечение, поддержка)
Выберите класс: 2
Введите имя: Никромант
Маг Никромант добавлен в партию!

Персонаж 3:
1. Воин (сильный, много HP)
2. Маг (интеллект, магические атаки)
3. Лекарь (лечение, поддержка)
Выберите класс: 3
Введите имя: Байрон
Лекарь Байрон добавлен в партию!

Выберите сложность (1-легкая, 2-нормальная, 3-сложная): 
1

Введите seed для случайной генерации (или оставьте пустым): 999
Установлен seed: 999
Создан босс: Владыка Бездны (Уровень 8)

Начало битвы!
Партия: ['Рыцарь', 'Никромант', 'Байрон']
Босс: Владыка Бездны (Уровень 8)
Начало битвы с раундами!

Начало битвы!
Партия: ['Рыцарь', 'Никромант', 'Байрон']
Босс: Влады